In [2]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/fasttext_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_4_feat.pkl
(159571, 6) (15

In [8]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': 3,
                    #'scale_pos_weight':0.9,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 100
            else:
                s_round = 50
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

done


In [9]:

lgb_res = simple_ens('lgb',10,233,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

# change to stratified
# all loss avg 0.0316412744167 0.0357232681944 all auc avg 0.994584962017 0.991293307537 pub 9866

0 fold: ls 0.0698851901581 0.0740509195698 auc 0.990655463053 0.988692556775
1 fold: ls 0.0681077620652 0.0785822918755 auc 0.991192109851 0.987372683109
2 fold: ls 0.0679412184374 0.0736855579113 auc 0.991324149915 0.989142707263
3 fold: ls 0.0654122429816 0.0755535041468 auc 0.992072852817 0.988375951989
4 fold: ls 0.0701588253184 0.0820682110627 auc 0.990406654887 0.986694763452
5 fold: ls 0.0698092711391 0.0745041428387 auc 0.990689925044 0.987905756248
6 fold: ls 0.0682949360885 0.0747374593949 auc 0.991155340324 0.988518528122
7 fold: ls 0.069456862445 0.0727452299348 auc 0.990852280727 0.988999488324
8 fold: ls 0.0671676903752 0.0782035788165 auc 0.991549867047 0.986690667882
9 fold: ls 0.0684445040213 0.0777410006759 auc 0.991095958188 0.987724809094
this class avg train 0.068467850303 avg val 0.0761871896227
this class auc train 0.991099460185 auc val 0.988011791226
0 fold: ls 0.0183116184221 0.0199285108817 auc 0.993908536304 0.992205500696
1 fold: ls 0.0181456613932 0.019564

In [12]:
# adj lr, feat_frac, bag_frac
for lr in [0.05]:
    for c1 in [0.6,0.7,0.8]:
        lgb_res = simple_ens('lgb',5,233,lr,c1)
        sample_submission = pd.read_csv("../input/sample_submission.csv")
        sample_submission[list_classes] = lgb_res
        fname = "../results/lgb_csv_fold5_{}_{}_s.gz".format(lr,c1)
        sample_submission.to_csv(fname, index=False, compression='gzip')
        print(fname)
        print(sample_submission.head())
        print('save done')
        break

# lr, feat_frac, bag_frac, bag_freq=40
# 0.03   0.7   0.7       all loss avg 0.0322530666235 0.0369762464226 all auc avg 0.994271044586 0.990285134782
# 0.03   0.7   0.8       all loss avg 0.0322971080147 0.0369559723174 all auc avg 0.994334369995 0.990306416634
# 0.03   0.8   0.7       all loss avg 0.0322229511884 0.0369948009383 all auc avg 0.994234840050 0.990368331216
# 0.03   0.8   0.8       all loss avg 0.0324206645596 0.0369651316109 all auc avg 0.994257933028 0.990295813203
# 0.05   0.7   0.7       all loss avg 0.0318163169078 0.0370403091733 all auc avg 0.994337086137 0.990254302965

# bag_freq = 3
# 0.05   0.6   0.6       all loss avg 0.0329345052647 0.0368498383873 all auc avg 0.9939819323 0.990664190969
# 0.05   0.6   0.7       all loss avg 0.0326017869132 0.0368044492709 all auc avg 0.994163898916 0.990696436189
# 0.05   0.6   0.8       all loss avg 0.0324589992208 0.0368163370811 all auc avg 0.994233438434 0.990618323557
# 0.05   0.7   0.6       all loss avg 0.0331541550186 0.0368670775083 all auc avg 0.993940708877 0.990597573572
# 0.05   0.7   0.7       all loss avg 0.0325753698442 0.0368285766463 all auc avg 0.994218398119 0.990663705936
# 0.05   0.7   0.8       all loss avg 0.0325779317956 0.0368281454058 all auc avg 0.994220915054 0.990630132284
# 0.05   0.8   0.6       all loss avg 0.0333336885412 0.0368817399664 all auc avg 0.993874933751 0.990630976344
# 0.05   0.8   0.7       all loss avg 0.0329593178033 0.0368484307605 all auc avg 0.994000316129 0.99065277921
# 0.05   0.8   0.8       all loss avg 0.0328682234385 0.0368555923112 all auc avg 0.994090826609 0.990708113059

# rm bagging
# 0.05   0.6             all loss avg 0.0328499864930 0.0368782748851 all auc avg 0.994090121326 0.990573017994   
# add 2 feats
# all loss avg 0.0319117001045 0.0360410792425 all auc avg 0.994541896459 0.991241823846
# rm pos_scale_weights, change l2 lambda to 1
# all loss avg 0.0316907179553 0.0359932860253 all auc avg 0.994616385099 0.991125111598

# add other feats
# all loss avg 0.031778552123 0.0359415940753 all auc avg 0.99463706224 0.991188429702

KeyboardInterrupt: 